In [1]:
import gym
import numpy as np
from keras import backend as K
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import TensorBoard
import random
import math
from time import time

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class ReplayBuffer:
    def __init__(self, max_size):
        self.max_size = max_size
        self.index = 0
        self.buffer = [None] * max_size
        self.count = 0

    """
    Add state, action, reward and next_state to replay buffer
    Use None for done state
    """
    def add(self, state, action, reward, next_state):
        self.buffer[self.index] = (state, action, reward, next_state)
        self.index = (self.index + 1) % self.max_size
        self.count = min(self.count + 1, self.max_size)

    def __len__(self):
        """
        Return current item count in replay buffer
        """
        return self.count

    def sample(self, count):
        """
        Sample `count` items from replay buffer
        Filter None items
        So result array may contain less items than count
        """
        count = min(count, self.count)
        samples = [s for s in self.buffer if s is not None]
        return random.sample(samples, count)

In [3]:
BUFFER_SIZE = 10000
BATCH_SIZE = 32

COPY_WEIGHTS_INTERVAL = 1000
    
MIN_EPSILON = 0.01
MAX_EPSILON = 1.0
LAMBDA = 0.001
    
GAMMA = 0.99

In [4]:
class Learner:
    def __init__(self, learner_type = 'dqn-v0', ob_dimen = 0, act_dimen = 0, buffer_size = BUFFER_SIZE):
        self.learner_type = learner_type
        self.ob_dimen = ob_dimen
        self.act_dimen = act_dimen
        self.replay_buffer = ReplayBuffer(buffer_size)
        self.act_counter = 0
        self.tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
        self.model = self._build_model()
        self.target_model = self._build_model()
        # set same weights initially
        self._update_target_network()
        
        # only compile model because we don't train on target network, we just use it to preduct q'
        self.model.compile(optimizer='adam', loss='mse')
        
    def _build_model(self):
        input_ob = Input(shape=(self.ob_dimen,), dtype='float32')
        fc1 = Dense(512, activation='relu')(input_ob)
        fc2 = Dense(64, activation='relu')(fc1)
        out = Dense(self.act_dimen)(fc2)
        model = Model(inputs=input_ob, outputs=out)
        return model
        
    def _predict(self, ob):
        return self.model.predict(np.asarray([ob]))
    
    def act(self, ob):
        self.act_counter += 1
        epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * math.exp(-LAMBDA * self.act_counter)
        rand = random.random()
        if len(self.replay_buffer) < BATCH_SIZE or rand < epsilon:
            return random.randrange(self.act_dimen)
        else:
            return np.argmax(self._predict(ob))
    
    def learn(self, ob, act, reward, next_ob):
        self.replay_buffer.add(ob, act, reward, next_ob)
        
        self._replay()
        
        if self.act_counter % COPY_WEIGHTS_INTERVAL == 0:
            self._update_target_network()
        
    def _replay(self):
        if len(self.replay_buffer) >= BATCH_SIZE:
            samples = self.replay_buffer.sample(BATCH_SIZE)
            empty_state = np.zeros(self.ob_dimen)
            obs = np.asarray([s[0] for s in samples])
            next_obs = np.asarray([s[3] if s[3] is not None else empty_state for s in samples])
            
            qs = self.model.predict(obs)
            next_qs = self.target_model.predict(next_obs)
            
            # if next state of i is terminal state(next_obs[i] is None), q value is: r
            # else, q value is: r + GAMMA * max(q(s'))
            for idx, sample in enumerate(samples):
                act = sample[1]
                reward = sample[2]
                done = sample[3] is None
                if done:
                    qs[idx][act] = reward
                else:
                    qs[idx][act] = reward + GAMMA * np.max(next_qs[idx])

            self.model.fit(obs, qs, verbose=0, callbacks=[self.tensorboard])
            
    def _update_target_network(self):
        print("Updating Network weights.")
        self.target_model.set_weights(self.model.get_weights())
    
    def save(self, path):
        self.model.save(path)
    
    def load(self, path):
        self.model = load_model(path)

In [5]:
class Agent:
    def __init__(self, env_name = 'CartPole-v0', learner = 'dqn-v0'):
        self.env = gym.make(env_name)
        self.ob_dimen = self.env.observation_space.shape[0]
        self.act_dimen = self.env.action_space.n
        self.learner = Learner(learner, self.ob_dimen, self.act_dimen)
    
    def start(self, epoch_count = 1000):
        for epc in range(1, epoch_count + 1):
            total_reward = 0
            done = False
            ob = self.env.reset()
            while not done:
                act = self.learner.act(ob)
                next_ob, reward, done, _ = self.env.step(act)
                
                self.learner.learn(ob, act, reward, None if done else next_ob)
                
                total_reward += reward
                ob = next_ob
            
            print("Epoch %s, Total Reward: %s" % (epc, total_reward))
            
    def save(self, path):
        self.learner.save(path)
    
    def load(self, path):
        self.learner.load(path)


In [6]:
#agent = Agent()
agent = Agent(env_name='MountainCar-v0')
agent.start(5000)
#agent.save('./cartpole-v0-dqn-v0.h5')
agent.save('./mountaincar-v0-dqn-v0.h5')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Updating Network weights.
Epoch 1, Total Reward: -200.0
Epoch 2, Total Reward: -200.0
Epoch 3, Total Reward: -200.0
Epoch 4, Total Reward: -200.0
Updating Network weights.
Epoch 5, Total Reward: -200.0
Epoch 6, Total Reward: -200.0
Epoch 7, Total Reward: -200.0
Epoch 8, Total Reward: -200.0
Epoch 9, Total Reward: -200.0
Updating Network weights.
Epoch 10, Total Reward: -200.0
Epoch 11, Total Reward: -200.0
Epoch 12, Total Reward: -200.0
Epoch 13, Total Reward: -200.0
Epoch 14, Total Reward: -200.0
Updating Network weights.
Epoch 15, Total Reward: -200.0
Epoch 16, Total Reward: -200.0
Epoch 17, Total Reward: -200.0
Epoch 18, Total Reward: -200.0
Epoch 19, Total Reward: -200.0
Updating Network weights.
Epoch 20, Total Reward: -200.0
Epoch 21, Total Reward: -200.0
Epoch 22, Total Reward: -200.0
Epoch 23, Total Reward: -200.0
Epoch 24, Total Reward: -200.0
Updating Network weights.
Epoch 25, 

Epoch 223, Total Reward: -200.0
Epoch 224, Total Reward: -193.0
Epoch 225, Total Reward: -92.0
Updating Network weights.
Epoch 226, Total Reward: -200.0
Epoch 227, Total Reward: -177.0
Epoch 228, Total Reward: -170.0
Epoch 229, Total Reward: -170.0
Epoch 230, Total Reward: -200.0
Epoch 231, Total Reward: -97.0
Updating Network weights.
Epoch 232, Total Reward: -177.0
Epoch 233, Total Reward: -200.0
Epoch 234, Total Reward: -153.0
Epoch 235, Total Reward: -180.0
Epoch 236, Total Reward: -200.0
Updating Network weights.
Epoch 237, Total Reward: -163.0
Epoch 238, Total Reward: -200.0
Epoch 239, Total Reward: -161.0
Epoch 240, Total Reward: -101.0
Epoch 241, Total Reward: -200.0
Epoch 242, Total Reward: -153.0
Epoch 243, Total Reward: -166.0
Updating Network weights.
Epoch 244, Total Reward: -200.0
Epoch 245, Total Reward: -87.0
Epoch 246, Total Reward: -158.0
Epoch 247, Total Reward: -200.0
Epoch 248, Total Reward: -200.0
Updating Network weights.
Epoch 249, Total Reward: -200.0
Epoch 250

Epoch 447, Total Reward: -155.0
Epoch 448, Total Reward: -200.0
Epoch 449, Total Reward: -200.0
Epoch 450, Total Reward: -181.0
Epoch 451, Total Reward: -163.0
Epoch 452, Total Reward: -160.0
Updating Network weights.
Epoch 453, Total Reward: -200.0
Epoch 454, Total Reward: -146.0
Epoch 455, Total Reward: -104.0
Epoch 456, Total Reward: -200.0
Epoch 457, Total Reward: -87.0
Epoch 458, Total Reward: -200.0
Updating Network weights.
Epoch 459, Total Reward: -200.0
Epoch 460, Total Reward: -162.0
Epoch 461, Total Reward: -157.0
Epoch 462, Total Reward: -200.0
Epoch 463, Total Reward: -168.0
Epoch 464, Total Reward: -200.0
Updating Network weights.
Epoch 465, Total Reward: -200.0
Epoch 466, Total Reward: -200.0
Epoch 467, Total Reward: -200.0
Epoch 468, Total Reward: -170.0
Epoch 469, Total Reward: -197.0
Updating Network weights.
Epoch 470, Total Reward: -200.0
Epoch 471, Total Reward: -170.0
Epoch 472, Total Reward: -88.0
Epoch 473, Total Reward: -87.0
Epoch 474, Total Reward: -200.0
Epo

Epoch 679, Total Reward: -146.0
Epoch 680, Total Reward: -147.0
Epoch 681, Total Reward: -123.0
Updating Network weights.
Epoch 682, Total Reward: -128.0
Epoch 683, Total Reward: -149.0
Epoch 684, Total Reward: -166.0
Epoch 685, Total Reward: -93.0
Epoch 686, Total Reward: -157.0
Epoch 687, Total Reward: -109.0
Epoch 688, Total Reward: -161.0
Epoch 689, Total Reward: -84.0
Updating Network weights.
Epoch 690, Total Reward: -103.0
Epoch 691, Total Reward: -134.0
Epoch 692, Total Reward: -149.0
Epoch 693, Total Reward: -106.0
Epoch 694, Total Reward: -146.0
Epoch 695, Total Reward: -151.0
Epoch 696, Total Reward: -104.0
Epoch 697, Total Reward: -104.0
Updating Network weights.
Epoch 698, Total Reward: -104.0
Epoch 699, Total Reward: -107.0
Epoch 700, Total Reward: -104.0
Epoch 701, Total Reward: -108.0
Epoch 702, Total Reward: -106.0
Epoch 703, Total Reward: -84.0
Epoch 704, Total Reward: -104.0
Epoch 705, Total Reward: -151.0
Epoch 706, Total Reward: -87.0
Updating Network weights.
Epoc

Epoch 916, Total Reward: -129.0
Epoch 917, Total Reward: -108.0
Epoch 918, Total Reward: -86.0
Epoch 919, Total Reward: -87.0
Epoch 920, Total Reward: -87.0
Epoch 921, Total Reward: -86.0
Epoch 922, Total Reward: -145.0
Epoch 923, Total Reward: -163.0
Updating Network weights.
Epoch 924, Total Reward: -108.0
Epoch 925, Total Reward: -130.0
Epoch 926, Total Reward: -107.0
Epoch 927, Total Reward: -100.0
Epoch 928, Total Reward: -110.0
Epoch 929, Total Reward: -106.0
Epoch 930, Total Reward: -143.0
Epoch 931, Total Reward: -142.0
Updating Network weights.
Epoch 932, Total Reward: -150.0
Epoch 933, Total Reward: -107.0
Epoch 934, Total Reward: -105.0
Epoch 935, Total Reward: -101.0
Epoch 936, Total Reward: -99.0
Epoch 937, Total Reward: -124.0
Epoch 938, Total Reward: -139.0
Epoch 939, Total Reward: -128.0
Epoch 940, Total Reward: -144.0
Updating Network weights.
Epoch 941, Total Reward: -136.0
Epoch 942, Total Reward: -134.0
Epoch 943, Total Reward: -108.0
Epoch 944, Total Reward: -105.0

...

Epoch 3931, Total Reward: -105.0
Epoch 3932, Total Reward: -88.0
Epoch 3933, Total Reward: -87.0
Updating Network weights.
Epoch 3934, Total Reward: -91.0
Epoch 3935, Total Reward: -105.0
Epoch 3936, Total Reward: -107.0
Epoch 3937, Total Reward: -104.0
Epoch 3938, Total Reward: -105.0
Epoch 3939, Total Reward: -86.0
Epoch 3940, Total Reward: -87.0
Epoch 3941, Total Reward: -106.0
Epoch 3942, Total Reward: -91.0
Epoch 3943, Total Reward: -102.0
Updating Network weights.
Epoch 3944, Total Reward: -91.0
Epoch 3945, Total Reward: -108.0
Epoch 3946, Total Reward: -105.0
Epoch 3947, Total Reward: -103.0
Epoch 3948, Total Reward: -199.0
Epoch 3949, Total Reward: -104.0
Epoch 3950, Total Reward: -105.0
Epoch 3951, Total Reward: -103.0
Updating Network weights.
Epoch 3952, Total Reward: -200.0
Epoch 3953, Total Reward: -108.0
Epoch 3954, Total Reward: -103.0
Epoch 3955, Total Reward: -102.0
Epoch 3956, Total Reward: -106.0
Epoch 3957, Total Reward: -113.0
Epoch 3958, Total Reward: -105.0
Epoch

Epoch 4162, Total Reward: -85.0
Epoch 4163, Total Reward: -91.0
Epoch 4164, Total Reward: -106.0
Epoch 4165, Total Reward: -108.0
Updating Network weights.
Epoch 4166, Total Reward: -96.0
Epoch 4167, Total Reward: -108.0
Epoch 4168, Total Reward: -87.0
Epoch 4169, Total Reward: -113.0
Epoch 4170, Total Reward: -95.0
Epoch 4171, Total Reward: -104.0
Epoch 4172, Total Reward: -105.0
Epoch 4173, Total Reward: -104.0
Epoch 4174, Total Reward: -100.0
Epoch 4175, Total Reward: -107.0
Updating Network weights.
Epoch 4176, Total Reward: -104.0
Epoch 4177, Total Reward: -109.0
Epoch 4178, Total Reward: -94.0
Epoch 4179, Total Reward: -108.0
Epoch 4180, Total Reward: -110.0
Epoch 4181, Total Reward: -104.0
Epoch 4182, Total Reward: -86.0
Epoch 4183, Total Reward: -102.0
Epoch 4184, Total Reward: -105.0
Epoch 4185, Total Reward: -88.0
Updating Network weights.
Epoch 4186, Total Reward: -86.0
Epoch 4187, Total Reward: -88.0
Epoch 4188, Total Reward: -109.0
Epoch 4189, Total Reward: -104.0
Epoch 41

Epoch 4412, Total Reward: -99.0
Epoch 4413, Total Reward: -111.0
Updating Network weights.
Epoch 4414, Total Reward: -161.0
Epoch 4415, Total Reward: -91.0
Epoch 4416, Total Reward: -90.0
Epoch 4417, Total Reward: -107.0
Epoch 4418, Total Reward: -109.0
Epoch 4419, Total Reward: -106.0
Epoch 4420, Total Reward: -105.0
Epoch 4421, Total Reward: -164.0
Epoch 4422, Total Reward: -106.0
Updating Network weights.
Epoch 4423, Total Reward: -104.0
Epoch 4424, Total Reward: -108.0
Epoch 4425, Total Reward: -110.0
Epoch 4426, Total Reward: -106.0
Epoch 4427, Total Reward: -105.0
Epoch 4428, Total Reward: -105.0
Epoch 4429, Total Reward: -91.0
Epoch 4430, Total Reward: -97.0
Epoch 4431, Total Reward: -106.0
Epoch 4432, Total Reward: -87.0
Updating Network weights.
Epoch 4433, Total Reward: -112.0
Epoch 4434, Total Reward: -88.0
Epoch 4435, Total Reward: -109.0
Epoch 4436, Total Reward: -119.0
Epoch 4437, Total Reward: -95.0
Epoch 4438, Total Reward: -107.0
Epoch 4439, Total Reward: -109.0
Epoch 

Epoch 4643, Total Reward: -111.0
Epoch 4644, Total Reward: -90.0
Epoch 4645, Total Reward: -111.0
Epoch 4646, Total Reward: -106.0
Epoch 4647, Total Reward: -89.0
Epoch 4648, Total Reward: -84.0
Epoch 4649, Total Reward: -85.0
Epoch 4650, Total Reward: -100.0
Updating Network weights.
Epoch 4651, Total Reward: -94.0
Epoch 4652, Total Reward: -166.0
Epoch 4653, Total Reward: -108.0
Epoch 4654, Total Reward: -111.0
Epoch 4655, Total Reward: -87.0
Epoch 4656, Total Reward: -108.0
Epoch 4657, Total Reward: -109.0
Epoch 4658, Total Reward: -120.0
Epoch 4659, Total Reward: -91.0
Updating Network weights.
Epoch 4660, Total Reward: -153.0
Epoch 4661, Total Reward: -151.0
Epoch 4662, Total Reward: -89.0
Epoch 4663, Total Reward: -86.0
Epoch 4664, Total Reward: -104.0
Epoch 4665, Total Reward: -115.0
Epoch 4666, Total Reward: -109.0
Epoch 4667, Total Reward: -95.0
Epoch 4668, Total Reward: -85.0
Updating Network weights.
Epoch 4669, Total Reward: -93.0
Epoch 4670, Total Reward: -90.0
Epoch 4671,

Epoch 4875, Total Reward: -120.0
Epoch 4876, Total Reward: -109.0
Epoch 4877, Total Reward: -95.0
Epoch 4878, Total Reward: -104.0
Epoch 4879, Total Reward: -93.0
Epoch 4880, Total Reward: -86.0
Epoch 4881, Total Reward: -102.0
Epoch 4882, Total Reward: -108.0
Epoch 4883, Total Reward: -111.0
Epoch 4884, Total Reward: -102.0
Updating Network weights.
Epoch 4885, Total Reward: -105.0
Epoch 4886, Total Reward: -86.0
Epoch 4887, Total Reward: -104.0
Epoch 4888, Total Reward: -97.0
Epoch 4889, Total Reward: -103.0
Epoch 4890, Total Reward: -107.0
Epoch 4891, Total Reward: -102.0
Epoch 4892, Total Reward: -83.0
Epoch 4893, Total Reward: -98.0
Epoch 4894, Total Reward: -102.0
Updating Network weights.
Epoch 4895, Total Reward: -87.0
Epoch 4896, Total Reward: -88.0
Epoch 4897, Total Reward: -106.0
Epoch 4898, Total Reward: -86.0
Epoch 4899, Total Reward: -103.0
Epoch 4900, Total Reward: -194.0
Epoch 4901, Total Reward: -105.0
Epoch 4902, Total Reward: -91.0
Epoch 4903, Total Reward: -97.0
Upd